In [109]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import sys
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import Dataset


In [85]:
batch_size=32
device='cpu'
lr=0.0001

In [622]:
class env():
  metadata = {'render_modes': ['human']}

  actions_set = {
                # increase channels with kernel 3
                'conv=channel_factor:2,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:4,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:8,kernel_size:3,stride:1,padding:0-',
                #'conv=channel_factor:16,kernel_size:3,stride:1,padding:0-',
                # increase channels with kernel 5
                'conv=channel_factor:2,kernel_size:5,stride:1,padding:0-',
                'conv=channel_factor:4,kernel_size:5,stride:1,padding:0-',
                'conv=channel_factor:8,kernel_size:5,stride:1,padding:0-',
                #'conv=channel_factor:16,kernel_size:5,stride:1,padding:0-',
                # increase channels with kernel 7
                'conv=channel_factor:2,kernel_size:7,stride:1,padding:0-',
                'conv=channel_factor:4,kernel_size:7,stride:1,padding:0-',
                'conv=channel_factor:8,kernel_size:7,stride:1,padding:0-',
                #'conv=channel_factor:16,kernel_size:7,stride:1,padding:0-',

                # decrease channels with kernel 3
                #'conv=channel_factor:0.1,kernel_size:3,stride:1,padding:0-',
                #'conv=channel_factor:0.2,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:0.4,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:0.8,kernel_size:3,stride:1,padding:0-',
                # decrease channels with kernel 5
                #'conv=channel_factor:0.1,kernel_size:5,stride:1,padding:0-',
                #'conv=channel_factor:0.2,kernel_size:5,stride:1,padding:0-',
                'conv=channel_factor:0.4,kernel_size:5,stride:1,padding:0-',
                'conv=channel_factor:0.8,kernel_size:5,stride:1,padding:0-',
                # decrease channels with kernel 7
                #'conv=channel_factor:0.1,kernel_size:3,stride:1,padding:0-',
                #'conv=channel_factor:0.2,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:0.4,kernel_size:3,stride:1,padding:0-',
                'conv=channel_factor:0.8,kernel_size:3,stride:1,padding:0-',


                 'batchnorm=eps:0.00001-',

                 'avgpool=kernel_size:2,stride:2,padding:0-',

                 'maxpool=kernel_size:2,stride:2,padding:0-',


                 'dropout=p:0.1-',
                 'dropout=p:0.2-',
                 'dropout=p:0.3-',
                 'dropout=p:0.4-',
  }

  NN_CREATE_SUCCESS_REWARD = 5
  NN_CREATE_NOT_SUCCESS_PENALTY = -5


  def __init__(self, render_mode=None):
      self.max_its = 10
      self.current_it = 1
      self.episode_reward = 0
      #self.last_layer_shape = None

      self.ENV_PARAMS = {
          'iteration': 1,
          'max_its': 10,
          'episode_reward': 0,
          'actions_amount': len(self.actions_set),
      }

      self.NN_PARAMS = {
          'train_epochs': 10,
          'last_nets_metrics_memory_len': 10,
          'max_layers_amount': 10,
          'min_layers_amount': 3,
          'in_channels': 1,
      }
      self.NN_PARAMS['metrics'] = {'valid_losses': np.array([0] * self.NN_PARAMS['train_epochs']),
                      'train_losses': np.array([0] * self.NN_PARAMS['train_epochs']),
                     }


      self.Net = self.NN(self.NN_PARAMS['in_channels'])
      self.train_dataloader = None
      self.train_dataloader = None
      self.optimizer_class = None
      self.optimizer = None
      self.criterion = None

      self.nngenerator = self.nnGenerator()

      self.last_obs = None

      self.action_space = spaces.Text(
          max_length=self.NN_PARAMS['max_layers_amount'],
          min_length=self.NN_PARAMS['min_layers_amount'],
          charset=self.actions_set)

      self.observation_space = spaces.Dict(
          {
          'last_nets_metrics_memory': spaces.Box(
              low=0,
              high=1,
              shape=(self.NN_PARAMS['last_nets_metrics_memory_len'],
                     self.NN_PARAMS['train_epochs'],
                     len(self.NN_PARAMS['metrics'].keys()),
                     )),

          }
      )
      self.statistics = {
        'episode_rewards': [],
        'global_rewards': [],
        'made_steps': [],
      }

      self.seed()
      assert render_mode is None or render_mode in self.metadata["render_modes"]
      self.render_mode = render_mode


  class NN(nn.Module):

    def __init__(self, in_channels):
      super().__init__()

      self.backbone = nn.Sequential()
      self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
      x = self.backbone(x)
      out = self.softmax(x)
      return out

    def __call__(self, x):
      return self.forward(x)

  def set_train_dataloader(self, train_dataloader):
    self.train_dataloader = train_dataloader

  def set_valid_dataloader(self, valid_dataloader):
    self.valid_dataloader = valid_dataloader

  def set_criterion(self, criterion):
    self.criterion = criterion

  def set_optimizer(self, optimizer):
    self.optimizer_class = optimizer

  def train(self):
    train_losses = []
    valid_losses = []
    # TODO calculate metrics and return them after train
    def CalcValLoss(self):
        with torch.no_grad():
            losses = []
            for X, Y in self.valid_dataloader:
                X = X.float().to(device)
                Y = Y.float().to(device)
                preds = self.Net(X)
                preds, _ = torch.max(preds,1)
                loss = self.criterion(preds,Y)
                losses.append(loss.item())
            print("Valid Loss : {:.6f}".format(torch.tensor(losses).mean()))
            valid_losses.append(torch.tensor(losses).mean())

    for i in range(1, self.NN_PARAMS['train_epochs']):
        losses = []
        for X, Y in tqdm(self.train_dataloader):
            X = X.float().to(device)
            Y = Y.float().to(device)
            preds = self.Net(X)
            preds, _ = torch.max(preds,1)
            loss = self.criterion(preds, Y)
            losses.append(loss.item())

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        print("Train Loss : {:.6f}".format(torch.tensor(losses).mean()))
        train_losses.append(torch.tensor(losses).mean())

    # update all required NN metrics
    self.NN_PARAMS['metrics']['train_losses'] = train_losses
    self.NN_PARAMS['metrics']['valid_losses'] = valid_losses

  class nnGenerator():
    def __init__(self):
      self.text_layers_dict = dict({})
      self.nn_len = -1

    def parse_action(self, action):
      print(action)
      if action[-1] == '-':
        action = action[:-1]
      text_layers = action.split('-')
      id = 0
      for text_layer in text_layers:
        tmp = text_layer.split('=')
        layer_name, layer_params = tmp[0], tmp[1].split(',')
        layer_params_dict = dict({})
        for param in layer_params:
          param = param.split(':')
          param_name, param_value = param[0], param[1]
          layer_params_dict[param_name] = param_value
        self.text_layers_dict[layer_name + str(id)] = (id, layer_params_dict)
        id += 1
      print(self.text_layers_dict)

    def get_text_layers_dict(self):
      return self.text_layers_dict

    def get_nn_len(self):
      return self.nn_len

    def conv_output_shape(self, h, w, kernel_size=1, stride=1, pad=0, dilation=1):
      h = math.floor( ((h + (2 * pad) - ( dilation * (kernel_size - 1) ) - 1 )/ stride) + 1)
      w = math.floor( ((w + (2 * pad) - ( dilation * (kernel_size - 1) ) - 1 )/ stride) + 1)
      return h, w

    def generateNN(self, n_classes, test_batch): # -> nn.Sequential
      # returns success_state, backbone and classifier of nn, nn.Sequential
      # if impossible to create nn, success_state = False
      success_state = False
      # TODO

      # update self.nn_len if created correctly

      backbone = nn.Sequential()
      classifier = nn.Sequential()
      optimizer = None
      try:
        data_shape = np.array(test_batch).shape # [B, C, H, W]
        last_shape = data_shape
        for layer_name in self.text_layers_dict.keys():
            layer = None
            layer_params= self.text_layers_dict[layer_name][1] # [0] - id
            if layer_name.find('conv') >= 0:
              # 'conv=channel_factor:2,kernel_size:5,stride:1,padding:0-',
              kernel_size = int(layer_params['kernel_size'])
              channel_factor = float(layer_params['channel_factor'])
              stride = int(layer_params['stride'])
              padding = int(layer_params['padding'])
              dilation = 1

              activation = nn.ReLU(inplace=True)

              in_chan = last_shape[1]
              out_chan = max(math.floor(in_chan * channel_factor), 1)

              backbone.append(nn.Conv2d(in_chan, out_chan, kernel_size, stride, padding, dilation))
              backbone.append(activation)

              h, w = self.conv_output_shape(last_shape[2], last_shape[3], kernel_size, stride, padding, dilation)
              last_shape = (last_shape[0], out_chan, h, w)

            elif layer_name.find('batchnorm') >= 0:
              eps = float(layer_params['eps'])
              in_chan = last_shape[1]
              backbone.append(nn.BatchNorm2d(in_chan, eps))

            elif layer_name.find('avgpool') >= 0:
              # 'avgpool=kernel_size:2,stride:2,padding:0-',
              kernel_size = int(layer_params['kernel_size'])
              stride = int(layer_params['stride'])
              padding = int(layer_params['padding'])
              dilation = 1
              out_chan = last_shape[1]
              backbone.append(nn.AvgPool2d(kernel_size, stride, padding))

              h, w = self.conv_output_shape(last_shape[2], last_shape[3], kernel_size, stride, padding, dilation)
              last_shape = (last_shape[0], out_chan, h, w)


            elif layer_name.find('maxpool') >= 0:
              # 'maxpool=kernel_size:2,stride:2,padding:0-',
              kernel_size = int(layer_params['kernel_size'])
              stride = int(layer_params['stride'])
              padding = int(layer_params['padding'])
              dilation = 1

              backbone.append(nn.MaxPool2d(kernel_size, stride, padding))

              h, w = self.conv_output_shape(last_shape[2], last_shape[3], kernel_size, stride, padding, dilation)
              last_shape = (last_shape[0], out_chan, h, w)

            elif layer_name.find('dropout') >= 0:
              p = float(layer_params['p'])
              backbone.append(nn.Dropout2d(p))
        linear_in_shape = last_shape[1] * last_shape[2] * last_shape[3]
        classifier = nn.Linear(linear_in_shape, n_classes)
        success_state = True
        print('NN build successfull!')
      except Exception as e:
        print('NN build failed!')
        print(str(e))
      net = nn.Sequential()
      net.append(backbone)
      net.append(nn.Flatten(start_dim=1))
      net.append(classifier)
      self.text_layers_dict = dict({})
      return success_state, net


  def seed(self, seed=None):
      self.np_random, seed = seeding.np_random(seed)
      return [seed]


  def calc_reward(self, nn_created_correctly_flag, nn_len, last_train_metrics):
    reward = 0
    # reward for decreasing nn depth
    optimal_depth_reward = 0
    # reward by metrics
    metrics_optimization_reward = 0
    # reward for successfull nn creation
    creation_successfull_reward = 0
    if nn_created_correctly_flag == True:
      # do not reward agent if creation is not succeed
      creation_successfull_reward += self.NN_CREATE_SUCCESS_REWARD
      # TODO
      metrics_optimization_reward += 1
      # TODO
      optimal_depth_reward

    else:
      creation_successfull_reward += self.NN_CREATE_NOT_SUCCESS_PENALTY

    reward += optimal_depth_reward
    reward += metrics_optimization_reward
    reward += creation_successfull_reward

    return reward

  def get_test_batch(self):
    batch = None
    for  b, _ in train_dataloader:
      batch = b
    return batch

  def create_obs(self):
    # TODO create new observation using
    # self.NN_PARAMS['metrics'] and self.last_obs
    obs = np.zeros((
              self.NN_PARAMS['last_nets_metrics_memory_len'],
              self.NN_PARAMS['train_epochs'],
              len(self.NN_PARAMS['metrics'].keys()),
             ) )
    return obs

  def step(self, action):
      reward = 0
      done = False
      info = {} # You may need to add some extra information

      '''
      1) parse action
      2) generate NN
      3) (optionally) update optimizer, prepare for training
      4) train NN, collect metrics
      5) calculate reward
      6) collect statistics
      7) create new observation
      8) return obs, reward, done, info
      '''
      reward = 0

      self.nngenerator.parse_action(action)
      test_b = self.get_test_batch()
      success_state, net = self.nngenerator.generateNN(n_classes=10, test_batch=test_b)
      # def generateNN(self, lr, n_classes, optimizer_class, test_batch):
      if success_state == True: # NN created_correctly
        self.Net.backbone = net
        display(self.Net)
        self.optimizer = self.optimizer_class(self.Net.parameters(), lr=0.001)
        self.train()


      reward = self.calc_reward(success_state,
                                self.nngenerator.get_nn_len(),
                                self.NN_PARAMS['metrics'],
                                )

      # TODO generate new observation
      new_obs = self.create_obs()

      current_obs = {
          'last_nets_metrics_memory': new_obs
          }

      self.last_obs = new_obs

      self.episode_reward = reward
      self.current_it += 1
      return current_obs, reward, done, info


  def reset(self):
      '''
      Reset the env
      '''
      # clearify all changable vars
      self.Net = self.NN(10)
      # self.criterion = None
      # may be it's better to set random number over zeros
      current_obs = {
          'last_nets_metrics_memory': np.zeros((
              self.NN_PARAMS['last_nets_metrics_memory_len'],
              self.NN_PARAMS['train_epochs'],
              len(self.NN_PARAMS['metrics'].keys()),
             ) )
          }
      self.last_obs = current_obs
      self.episode_reward = 0
      self.current_it = 1

      self.statistics['episode_rewards'] = []
      self.statistics['made_steps'] = []

      return current_obs



  def render(self, mode=None):
    #TODO visualization
    pass


  def close(self):
      pass

In [623]:
e = env()

In [624]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [625]:
train_samples = np.expand_dims(np.array(train_data.data), axis=1)[:5000]

In [626]:
train_labels = np.array(train_data.targets)[:5000]

In [627]:

class myDataset(Dataset):
    def __init__(self, X, y):

      self.X = X
      self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x_ = self.X[idx]
        y_ = self.y[idx]
        return x_, y_

In [628]:
dataset = myDataset(X=train_samples, y=train_labels)

In [629]:
train_set, valid_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

In [630]:

train_dataloader = torch.utils.data.DataLoader(
  train_set,
  batch_size=batch_size,
  shuffle=True,
  drop_last=True)

valid_dataloader = torch.utils.data.DataLoader(
  valid_set,
  batch_size=batch_size,
  drop_last=True,
  shuffle=True)

In [631]:
e.set_train_dataloader(train_dataloader)
e.set_valid_dataloader(valid_dataloader)

In [632]:
e.set_criterion(torch.nn.CrossEntropyLoss())
e.set_optimizer(torch.optim.SGD)

In [633]:
e.reset();

In [634]:
t = spaces.Text(max_length=8, min_length=7,charset=e.actions_set)
action = t.sample()
print(action)

conv=channel_factor:0.8,kernel_size:5,stride:1,padding:0-conv=channel_factor:2,kernel_size:3,stride:1,padding:0-conv=channel_factor:0.4,kernel_size:3,stride:1,padding:0-dropout=p:0.4-conv=channel_factor:2,kernel_size:7,stride:1,padding:0-conv=channel_factor:8,kernel_size:7,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-maxpool=kernel_size:2,stride:2,padding:0-


In [635]:
e.reset();

In [636]:
r = e.step(action)

conv=channel_factor:0.8,kernel_size:5,stride:1,padding:0-conv=channel_factor:2,kernel_size:3,stride:1,padding:0-conv=channel_factor:0.4,kernel_size:3,stride:1,padding:0-dropout=p:0.4-conv=channel_factor:2,kernel_size:7,stride:1,padding:0-conv=channel_factor:8,kernel_size:7,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-maxpool=kernel_size:2,stride:2,padding:0-
{'conv0': (0, {'channel_factor': '0.8', 'kernel_size': '5', 'stride': '1', 'padding': '0'}), 'conv1': (1, {'channel_factor': '2', 'kernel_size': '3', 'stride': '1', 'padding': '0'}), 'conv2': (2, {'channel_factor': '0.4', 'kernel_size': '3', 'stride': '1', 'padding': '0'}), 'dropout3': (3, {'p': '0.4'}), 'conv4': (4, {'channel_factor': '2', 'kernel_size': '7', 'stride': '1', 'padding': '0'}), 'conv5': (5, {'channel_factor': '8', 'kernel_size': '7', 'stride': '1', 'padding': '0'}), 'conv6': (6, {'channel_factor': '4', 'kernel_size': '3', 'stride': '1', 'padding': '0'}), 'maxpool7': (7, {'kernel_size': '2

NN(
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(2, 1, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU(inplace=True)
      (6): Dropout2d(p=0.4, inplace=False)
      (7): Conv2d(1, 2, kernel_size=(7, 7), stride=(1, 1))
      (8): ReLU(inplace=True)
      (9): Conv2d(2, 16, kernel_size=(7, 7), stride=(1, 1))
      (10): ReLU(inplace=True)
      (11): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1))
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=576, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)

100%|██████████| 125/125 [00:02<00:00, 62.19it/s]


Train Loss : 491.494354


100%|██████████| 125/125 [00:01<00:00, 63.10it/s]


...

Train Loss : 482.084290


100%|██████████| 125/125 [00:01<00:00, 63.04it/s]


Train Loss : 482.304779


100%|██████████| 125/125 [00:02<00:00, 62.42it/s]

Train Loss : 481.738403


In [262]:
e.train()

  0%|          | 0/125 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
# PyTorch 2D layers with parameters
# conv2d() + relu
# torch.nn.Conv2d(
#     in_channels,
#     out_channels,
#     kernel_size,
#     stride=1,
#     padding=0,
#     dilation=1,
#     groups=1,
#     bias=True,
#     padding_mode='zeros',
#     device=None,
#     dtype=None
#     )

# batchnorm2d()
# torch.nn.BatchNorm2d(
#     num_features,
#     eps=1e-05,
#     momentum=0.1,
#     #affine=True,
#     #track_running_stats=True,
#     #device=None,
#     #dtype=None
#     )

# avgpool2d()
# torch.nn.AvgPool2d(
#     kernel_size,
#     stride=None,
#     padding=0,
#     #ceil_mode=False,
#     #count_include_pad=True,
#     #divisor_override=None
#     )

# maxpool2d()
# torch.nn.MaxPool2d(
#     kernel_size,
#     stride=None,
#     padding=0,
#     dilation=1,
#     return_indices=False,
#     ceil_mode=False)

# dropout()
# torch.nn.Dropout2d(
#     p=0.5,
#     #inplace=False
#     )

In [120]:
t = spaces.Text(max_length=10, min_length=3,charset=e.actions_set)
action = t.sample()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
action = t.sample()

/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [ ]:
'conv=channel_factor:4,kernel_size:7,stride:1,padding:0-conv=channel_factor:0.2,kernel_size:5,stride:1,padding:0-conv=channel_factor:0.4,kernel_size:5,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-conv=channel_factor:2,kernel_size:7,stride:1,padding:0-batchnorm=eps:0.00001-maxpool=kernel_size:2,stride:2,padding:0-conv=channel_factor:0.8,kernel_size:3,stride:1,padding:0-'

In [122]:
action

'conv=channel_factor:4,kernel_size:7,stride:1,padding:0-conv=channel_factor:0.2,kernel_size:5,stride:1,padding:0-conv=channel_factor:0.4,kernel_size:5,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-conv=channel_factor:4,kernel_size:3,stride:1,padding:0-conv=channel_factor:2,kernel_size:7,stride:1,padding:0-batchnorm=eps:0.00001-maxpool=kernel_size:2,stride:2,padding:0-conv=channel_factor:0.8,kernel_size:3,stride:1,padding:0-'

In [73]:
class actionParser():
  def __init__(self):
    self.text_layers_dict = dict({})
    pass

  def parse_action(self, action):
    if action[-1] == '-':
      action = action[:-1]
    text_layers = action.split('-')
    for text_layer in text_layers:
      tmp = text_layer.split('=')
      layer_name, layer_params = tmp[0], tmp[1].split(',')
      layer_params_dict = dict({})
      for param in layer_params:
        param = param.split(':')
        param_name, param_value = param[0], param[1]
        layer_params_dict[param_name] = param_value
      self.text_layers_dict[layer_name] = layer_params_dict

  def get_text_layers_dict(self):
    return self.text_layers_dict

In [74]:
parser = actionParser()
parser.parse_action(action)
res = parser.get_text_layers_dict()
for r in res.keys():
  print(r, res[r])

batchnorm {'eps': '0.00001'}
maxpool {'kernel_size': '2', 'stride': '2', 'padding': '0'}
avgpool {'kernel_size': '3', 'stride': '2', 'padding': '0'}


In [110]:
s = nn.Sequential()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
s.append(nn.Dropout(p=0.2))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sequential(
  (0): Dropout(p=0.2, inplace=False)
)

In [113]:
s

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sequential(
  (0): Dropout(p=0.2, inplace=False)
)